In [1]:
import time
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import dgl
import networkx as nx
import torch
import torch.nn as nn

import utils
from gsp_utils.baselines_archs import GCNN_2L
from gsp_utils.baselines_models import NodeClassModel, GF_NodeClassModel
from gsp_utils.data import normalize_gso
from src.arch import GFGCN, GFGCNLayer, GFGCN_noh_Layer, GFGCN_Spows

SEED = 15
# PATH = 'results/diff_filters/'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

torch.manual_seed(SEED)

cuda:0


In [2]:
# Auxiliary functions
def compute_S_pows(S, K, device):
    N = S.shape[0]
    S_pows = torch.Tensor(torch.empty(K-1, N, N)).to(device)
    S_pows[0,:,:] = torch.Tensor(S).to(device)
    for k in range(1,K-1):
        S_pows[k,:,:] = S_pows[0,:,:] @ S_pows[k-1,:,:]

    return S_pows

In [3]:
# Dataset must be from DGL
dataset_name = 'CornellDataset'

A, feat, labels, n_class, masks = utils.get_data_dgl(dataset_name, dev=device,
                                                     verb=True)
N = A.shape[0]

Dataset: CornellDataset
Number of nodes: 183
Number of features: 1703
Shape of signals: torch.Size([183, 1703])
Number of classes: 5
Norm of A: 17.262676239013672
Max value of A: 1.0
Proportion of validation data: 0.32
Proportion of test data: 0.20
Node homophily: 0.11
Edge homophily: 0.13


In [4]:
# BEST PARAMETERS
## Reaining params
N_RUNS = 10
N_EPOCHS = 1000  # 500
LR = .05
WD = .01
DROPOUT = 0

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3
HID_DIM = 32

## Model params
NORM = True

IN_DIM = feat.shape[1]
OUT_DIM = n_class


ACT = nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss()

## Training paramerters

In [5]:
EXPS = [
        {'epochs': 500, 'lr': .1, 'wd': 5e-2},
        {'epochs': 500, 'lr': .05, 'wd': 5e-2},
        {'epochs': 500, 'lr': .01, 'wd': 5e-2},
        {'epochs': 500, 'lr': .005, 'wd': 5e-2},

        {'epochs': 500, 'lr': .01, 'wd': 1e-1},
        {'epochs': 500, 'lr': .01, 'wd': 1e-2},
        {'epochs': 500, 'lr': .01, 'wd': 1e-3},
        {'epochs': 500, 'lr': .05, 'wd': 1e-1},
        {'epochs': 500, 'lr': .05, 'wd': 1e-2},
        {'epochs': 500, 'lr': .05, 'wd': 1e-3},


        {'epochs': 1000, 'lr': .05, 'wd': 1e-2},
        {'epochs': 5000, 'lr': .01, 'wd': 1e-3},
        {'epochs': 5000, 'lr': .05, 'wd': 1e-2},
        ]

best_accs = np.zeros((len(EXPS), N_RUNS))
t_i = time.time()
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):        
        arch = GFGCN_Spows(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                           dropout=DROPOUT, norm=NORM, dev=device)
        S = torch.Tensor(A).to(device)

        model = GF_NodeClassModel(arch, S, K, masks, LOSS_FN, device=device)
        _, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'])

        best_accs[j,i] = np.max(acc["test"])

        print(f'-{i+1}: {exp["epochs"]}-{exp["lr"]}-{exp["wd"]}: {best_accs[j,i]:.3f}')

ellapsed_t = (time.time()-t_i)/60
print(f'----- {ellapsed_t:.2f} mins -----')

# Get table with results
mean_accs = best_accs.mean(axis=1)
std_accs = best_accs.std(axis=1)
index_name = [f'{exp["epochs"]}-{exp["lr"]}-{exp["wd"]}' for exp in EXPS]
table1 = DataFrame(np.vstack((mean_accs, std_accs)).T, columns=['mean accs', 'std'], index=index_name)

-1: 500-0.1-0.05: 0.649
-1: 500-0.05-0.05: 0.703
-1: 500-0.01-0.05: 0.622
-1: 500-0.005-0.05: 0.486
-1: 500-0.01-0.1: 0.514
-1: 500-0.01-0.01: 0.811
-1: 500-0.01-0.001: 0.784
-1: 500-0.05-0.1: 0.541
-1: 500-0.05-0.01: 0.838
-1: 500-0.05-0.001: 0.703
-1: 1000-0.05-0.01: 0.838
-1: 5000-0.01-0.001: 0.811
-1: 5000-0.05-0.01: 0.811
-2: 500-0.1-0.05: 0.649
-2: 500-0.05-0.05: 0.622
-2: 500-0.01-0.05: 0.541
-2: 500-0.005-0.05: 0.541
-2: 500-0.01-0.1: 0.541
-2: 500-0.01-0.01: 0.514
-2: 500-0.01-0.001: 0.784
-2: 500-0.05-0.1: 0.541


In [ ]:
table1

,mean accs,std
500-0.1-0.05,0.628108,0.066326
500-0.05-0.05,0.681081,0.086486
500-0.01-0.05,0.591351,0.078078
500-0.005-0.05,0.542703,0.062066
500-0.01-0.1,0.561081,0.041365
500-0.01-0.01,0.722162,0.109055
500-0.01-0.001,0.761081,0.084324
500-0.05-0.1,0.628108,0.075784
500-0.05-0.01,0.741622,0.105670
500-0.05-0.001,0.682162,0.122377


## Model params

In [ ]:
# h0, norm (norm, not norm, norm H), sep vs joint training, optimizers iters (W vs h)
EXPS = [{'epochs': 500, 'epochs_h': 1, 'epochs_W': 1, 'alt': True},
        {'epochs': 2000, 'epochs_h': 1, 'epochs_W': 1, 'alt': True},        
        {'epochs': 500, 'epochs_h': 1, 'epochs_W': 1, 'alt': False},

        {'epochs': 500, 'epochs_h': 10, 'epochs_W': 1, 'alt': True},
        {'epochs': 500, 'epochs_h': 25, 'epochs_W': 1, 'alt': True},
        {'epochs': 500, 'epochs_h': 1, 'epochs_W': 10, 'alt': True},
        {'epochs': 500, 'epochs_h': 1, 'epochs_W': 25, 'alt': True},
        {'epochs': 200, 'epochs_h': 1, 'epochs_W': 50, 'alt': True},

        {'epochs': 500, 'epochs_h': 10, 'epochs_W': 10, 'alt': True},
        {'epochs': 500, 'epochs_h': 25, 'epochs_W': 25, 'alt': True},
        {'epochs': 100, 'epochs_h': 50, 'epochs_W': 50, 'alt': True},]


best_accs2 = np.zeros((len(EXPS), N_RUNS))
t_i = time.time()
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):
        arch = GFGCN_Spows(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                           dropout=DROPOUT, norm=NORM, dev=device)
        S = torch.Tensor(A).to(device)

        if not exp['alt']:
            S_pows = compute_S_pows(A, K, device)
            model = NodeClassModel(arch, S_pows, masks, LOSS_FN, device=device)
            _, acc = model.train(feat, labels, exp['epochs'], LR, WD)
        else:
            model = GF_NodeClassModel(arch, S,  K, masks, LOSS_FN, device=device)
            _, acc = model.train(feat, labels, exp['epochs'], LR, WD, epochs_h=exp['epochs_h'],
                                 epochs_W=exp['epochs_W'])

        best_accs2[j,i] = np.max(acc["test"])

        print(f'-{i+1}: {exp["epochs_h"]}-{exp["epochs_W"]}-{exp["alt"]}: {best_accs2[j,i]:.3f}')

ellapsed_t = (time.time()-t_i)/60
print(f'----- {ellapsed_t:.2f} mins -----')

# Get table with results
mean_accs = best_accs2.mean(axis=1)
std_accs = best_accs2.std(axis=1)
index_name = [f'{exp["epochs"]}-{exp["epochs_h"]}-{exp["epochs_W"]}-{exp["alt"]}' for exp in EXPS]
table2 = DataFrame(np.vstack((mean_accs, std_accs)).T, columns=['mean accs', 'std'], index=index_name)

-1: 1-1-True: 0.784
-1: 1-1-True: 0.757
-1: 1-1-False: 0.270
-1: 10-1-True: 0.784
-1: 1-10-True: 0.811
-1: 1-25-True: 0.730
-1: 1-50-True: 0.757
-1: 10-10-True: 0.784
-1: 25-25-True: 0.811
-1: 50-50-True: 0.757
-2: 1-1-True: 0.811
-2: 1-1-True: 0.892
-2: 1-1-False: 0.270
-2: 10-1-True: 0.622
-2: 1-10-True: 0.595
-2: 1-25-True: 0.865
-2: 1-50-True: 0.730
-2: 10-10-True: 0.811
-2: 25-25-True: 0.838
-2: 50-50-True: 0.784
-3: 1-1-True: 0.541
-3: 1-1-True: 0.838
-3: 1-1-False: 0.270
-3: 10-1-True: 0.757
-3: 1-10-True: 0.784
-3: 1-25-True: 0.811
-3: 1-50-True: 0.730
-3: 10-10-True: 0.784
-3: 25-25-True: 0.757
-3: 50-50-True: 0.784
-4: 1-1-True: 0.703
-4: 1-1-True: 0.757
-4: 1-1-False: 0.270
-4: 10-1-True: 0.730
-4: 1-10-True: 0.649
-4: 1-25-True: 0.622
-4: 1-50-True: 0.568
-4: 10-10-True: 0.838
-4: 25-25-True: 0.811
-4: 50-50-True: 0.784
-5: 1-1-True: 0.838
-5: 1-1-True: 0.838
-5: 1-1-False: 0.270
-5: 10-1-True: 0.730
-5: 1-10-True: 0.757
-5: 1-25-True: 0.703
-5: 1-50-True: 0.568
-5: 10-10-T

In [ ]:
table2

,mean accs,std
200-1-1-True,0.709189,0.119233
5000-1-1-True,0.838919,0.046800
200-1-1-False,0.270270,0.000000
200-10-1-True,0.722162,0.109590
200-1-10-True,0.707027,0.096900
200-1-25-True,0.695135,0.123622
100-1-50-True,0.670270,0.103975
200-10-10-True,0.756757,0.084780
200-25-25-True,0.777297,0.070594
100-50-50-True,0.735135,0.095784


## Architecture design

In [ ]:
# layers, filter order, weightd
EXPS = [{'L': 2, 'K': 2, 'hid_dim': 16},
        {'L': 2, 'K': 3, 'hid_dim': 16},
        {'L': 2, 'K': 4, 'hid_dim': 16},

        {'L': 3, 'K': 2, 'hid_dim': 16},
        {'L': 4, 'K': 2, 'hid_dim': 16},
        {'L': 3, 'K': 3, 'hid_dim': 16},
        {'L': 4, 'K': 3, 'hid_dim': 16},

        {'L': 2, 'K': 2, 'hid_dim': 8},
        {'L': 2, 'K': 2, 'hid_dim': 32},
        {'L': 2, 'K': 2, 'hid_dim': 50},

        {'L': 2, 'K': 3, 'hid_dim': 8},
        {'L': 2, 'K': 3, 'hid_dim': 32},
        {'L': 2, 'K': 3, 'hid_dim': 50},

        {'L': 3, 'K': 2, 'hid_dim': 8},
        {'L': 3, 'K': 2, 'hid_dim': 32},
        {'L': 3, 'K': 2, 'hid_dim': 50},

        {'L': 3, 'K': 3, 'hid_dim': 8},
        {'L': 3, 'K': 3, 'hid_dim': 32},
        {'L': 3, 'K': 3, 'hid_dim': 50},
        ]

best_accs = np.zeros((len(EXPS), N_RUNS))
t_i = time.time()
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):
        arch = GFGCN_Spows(IN_DIM, exp['hid_dim'], OUT_DIM, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                           dropout=DROPOUT, norm=NORM, dev=device)
        S = torch.Tensor(A).to(device)

        model = GF_NodeClassModel(arch, S,  exp['K'], masks, LOSS_FN, device=device)
        _, acc = model.train(feat, labels, N_EPOCHS, LR, WD)

        best_accs[j,i] = np.max(acc["test"])

        print(f'-{i+1}: {exp["L"]}-{exp["K"]}-{exp["hid_dim"]}: {best_accs[j,i]:.3f}')

ellapsed_t = (time.time()-t_i)/60
print(f'----- {ellapsed_t:.2f} mins -----')

# Get table with results
mean_accs = best_accs.mean(axis=1)
std_accs = best_accs.std(axis=1)
index_name = [f'{exp["L"]}-{exp["K"]}-{exp["hid_dim"]}' for exp in EXPS]
table3 = DataFrame(np.vstack((mean_accs, std_accs)).T, columns=['mean accs', 'std'], index=index_name)

-1: 2-2-16: 0.892
-1: 2-3-16: 0.784
-1: 2-4-16: 0.595
-1: 3-2-16: 0.865
-1: 4-2-16: 0.568
-1: 3-3-16: 0.541
-1: 4-3-16: 0.541
-1: 2-2-8: 0.892
-1: 2-2-32: 0.838
-1: 2-2-50: 0.865
-1: 2-3-8: 0.811
-1: 2-3-32: 0.784
-1: 2-3-50: 0.568
-1: 3-2-8: 0.703
-1: 3-2-32: 0.865
-1: 3-2-50: 0.838
-1: 3-3-8: 0.784
-1: 3-3-32: 0.541
-1: 3-3-50: 0.892
-2: 2-2-16: 0.865
-2: 2-3-16: 0.838
-2: 2-4-16: 0.622
-2: 3-2-16: 0.892
-2: 4-2-16: 0.703
-2: 3-3-16: 0.703
-2: 4-3-16: 0.568
-2: 2-2-8: 0.865
-2: 2-2-32: 0.730
-2: 2-2-50: 0.865
-2: 2-3-8: 0.757
-2: 2-3-32: 0.865
-2: 2-3-50: 0.649
-2: 3-2-8: 0.811
-2: 3-2-32: 0.811
-2: 3-2-50: 0.865
-2: 3-3-8: 0.730
-2: 3-3-32: 0.811
-2: 3-3-50: 0.595
-3: 2-2-16: 0.892
-3: 2-3-16: 0.757
-3: 2-4-16: 0.730
-3: 3-2-16: 0.865
-3: 4-2-16: 0.838
-3: 3-3-16: 0.595
-3: 4-3-16: 0.541
-3: 2-2-8: 0.838
-3: 2-2-32: 0.892
-3: 2-2-50: 0.892
-3: 2-3-8: 0.676
-3: 2-3-32: 0.649
-3: 2-3-50: 0.784
-3: 3-2-8: 0.649
-3: 3-2-32: 0.919
-3: 3-2-50: 0.676
-3: 3-3-8: 0.730
-3: 3-3-32: 0.541
-3: 

In [ ]:
table3

,mean accs,std
2-2-16,0.858378,0.019157
2-3-16,0.859459,0.021622
2-4-16,0.848649,0.026481
3-2-16,0.851892,0.018911
4-2-16,0.787027,0.093099
3-3-16,0.838919,0.066852
4-3-16,0.805405,0.092998
2-2-8,0.835676,0.021513
2-2-32,0.862703,0.015135
2-2-50,0.869189,0.018219


## Nonlinearities and loss function

In [ ]:
# layers, filter order, weightd
EXPS = [{'act': nn.ReLU(), 'lact': nn.Softmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.ReLU(), 'lact': nn.Softmax(dim=1), 'loss': nn.NLLLoss()},

        {'act': nn.ReLU(), 'lact': nn.LogSoftmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.ReLU(), 'lact': nn.LogSoftmax(dim=1), 'loss': nn.NLLLoss()},
        {'act': nn.ReLU(), 'lact': nn.Identity(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.ReLU(), 'lact': nn.Identity(dim=1), 'loss': nn.NLLLoss()},

        {'act': nn.ELU(), 'lact': nn.Softmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.ELU(), 'lact': nn.Softmax(dim=1), 'loss': nn.NLLLoss()},
        {'act': nn.LeakyReLU(), 'lact': nn.Softmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.LeakyReLU(), 'lact': nn.Softmax(dim=1), 'loss': nn.NLLLoss()},

        {'act': nn.Identity(), 'lact': nn.Softmax(dim=1), 'loss': nn.CrossEntropyLoss()},
        {'act': nn.Identity(), 'lact': nn.Softmax(dim=1), 'loss': nn.NLLLoss()},
        {'act': nn.Identity(), 'lact': nn.Identity(dim=1), 'loss': nn.CrossEntropyLoss()},]

best_accs4 = np.zeros((len(EXPS), N_RUNS))
t_i = time.time()
for i in range(N_RUNS):
    for j, exp in enumerate(EXPS):
        arch = GFGCN_Spows(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=exp['act'], last_act=exp['lact'],
                           dropout=DROPOUT, norm=NORM, dev=device)
        S = torch.Tensor(A).to(device)

        model = GF_NodeClassModel(arch, S,  K, masks, exp['loss'], device=device)
        _, acc = model.train(feat, labels, N_EPOCHS, LR, WD)

        best_accs4[j,i] = np.max(acc["test"])

        print(f'-{i+1}: {exp["act"]}-{exp["lact"]}-{exp["loss"]}: {best_accs4[j,i]:.3f}')

ellapsed_t = (time.time()-t_i)/60
print(f'----- {ellapsed_t:.2f} mins -----')

# Get table with results
mean_accs = best_accs4.mean(axis=1)
std_accs = best_accs4.std(axis=1)
index_name = [f'{exp["act"]}-{exp["lact"]}-{exp["loss"]}' for exp in EXPS]
table4 = DataFrame(np.vstack((mean_accs, std_accs)).T, columns=['mean accs', 'std'], index=index_name)

-1: ReLU()-Softmax(dim=1)-CrossEntropyLoss(): 0.784
-1: ReLU()-Softmax(dim=1)-NLLLoss(): 0.865
-1: ReLU()-LogSoftmax(dim=1)-CrossEntropyLoss(): 0.730
-1: ReLU()-LogSoftmax(dim=1)-NLLLoss(): 0.730
-1: ReLU()-Identity()-CrossEntropyLoss(): 0.811
-1: ReLU()-Identity()-NLLLoss(): 0.541
-1: ELU(alpha=1.0)-Softmax(dim=1)-CrossEntropyLoss(): 0.838
-1: ELU(alpha=1.0)-Softmax(dim=1)-NLLLoss(): 0.865
-1: LeakyReLU(negative_slope=0.01)-Softmax(dim=1)-CrossEntropyLoss(): 0.649
-1: LeakyReLU(negative_slope=0.01)-Softmax(dim=1)-NLLLoss(): 0.703
-1: Identity()-Softmax(dim=1)-CrossEntropyLoss(): 0.541
-1: Identity()-Softmax(dim=1)-NLLLoss(): 0.838
-1: Identity()-Identity()-CrossEntropyLoss(): 0.838
-2: ReLU()-Softmax(dim=1)-CrossEntropyLoss(): 0.838
-2: ReLU()-Softmax(dim=1)-NLLLoss(): 0.865
-2: ReLU()-LogSoftmax(dim=1)-CrossEntropyLoss(): 0.730
-2: ReLU()-LogSoftmax(dim=1)-NLLLoss(): 0.703
-2: ReLU()-Identity()-CrossEntropyLoss(): 0.811
-2: ReLU()-Identity()-NLLLoss(): 0.514
-2: ELU(alpha=1.0)-Softma

In [ ]:
table4

,mean accs,std
ReLU()-Softmax(dim=1)-CrossEntropyLoss(),0.802703,0.049909
ReLU()-Softmax(dim=1)-NLLLoss(),0.794595,0.092209
ReLU()-LogSoftmax(dim=1)-CrossEntropyLoss(),0.783784,0.040088
ReLU()-LogSoftmax(dim=1)-NLLLoss(),0.772973,0.048649
ReLU()-Identity()-CrossEntropyLoss(),0.775676,0.054122
ReLU()-Identity()-NLLLoss(),0.459459,0.065090
ELU(alpha=1.0)-Softmax(dim=1)-CrossEntropyLoss(),0.821622,0.055652
ELU(alpha=1.0)-Softmax(dim=1)-NLLLoss(),0.748649,0.126219
LeakyReLU(negative_slope=0.01)-Softmax(dim=1)-CrossEntropyLoss(),0.756757,0.098935
LeakyReLU(negative_slope=0.01)-Softmax(dim=1)-NLLLoss(),0.781081,0.077817


# Comparing Models

In [ ]:
EXPS = [
        {'name': 'Kipf', 'norm': 'none'},
        {'name': 'Kipf', 'norm': 'both'},

        {'name': 'A-GCNN', 'norm': False},
        {'name': 'A-GCNN', 'norm': True},

        {'name': 'H-GCNN', 'norm': False},
        {'name': 'H-GCNN', 'norm': True},

        {'name': 'W-GCN-A', 'norm': False},
        {'name': 'W-GCN-A', 'norm': True},
]

In [ ]:
best_accs = np.zeros((len(EXPS), N_RUNS))
for i in range(N_RUNS):
    print(f'- RUN: {i+1}')
    for j, exp in enumerate(EXPS):
        # t_i = time.time()
        if exp['name'] == 'Kipf':
            arch = GCNN_2L(IN_DIM, HID_DIM, OUT_DIM, act=ACT, last_act=LAST_ACT,
                           dropout=DROPOUT, norm=exp['norm'])
            S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)
            
        elif exp['name'] == 'A-GCNN':
            arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
            dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=h0)
            if exp['norm']:
                S = torch.Tensor(normalize_gso(A, 'both')).to(device)
            else:
                S = torch.Tensor(A).to(device)

        elif exp['name'] == 'H-GCNN':
            arch = GFGCN_Spows(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'], dev=device)
            S = torch.Tensor(A).to(device)

        elif exp['name'] == 'W-GCN-A':
            arch = GFGCN(IN_DIM, HID_DIM, OUT_DIM, N_LAYERS, K, act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
            if exp['norm']:
                S = torch.Tensor(normalize_gso(A, 'both')).to(device)
            else:
                S = torch.Tensor(A).to(device)  
            
        else:
            raise Exception(f'ERROR: Unknown architecture: {exp["name"]}')

        if exp['name'] in ['Kipf', 'W-GCN-A']:
            model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
        else:
            model = GF_NodeClassModel(arch, S, K, masks, LOSS_FN, device=device)

        loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)

        best_accs[j,i] = np.max(acc["test"])

        print(f'\t{exp["name"]}-{exp["norm"]}: acc = {best_accs[j,i]:.3f}')


# Print results
mean_accs = best_accs.mean(axis=1)
med_accs = np.median(best_accs, axis=1)
std_accs = best_accs.std(axis=1)

index_name = [f'{exp["name"]}-{exp["norm"]}' for exp in EXPS]
data = np.vstack((mean_accs, med_accs, std_accs)).T
table_comp2 = DataFrame(data, columns=['mean accs', 'med accs', 'std'], index=index_name)
        

- RUN: 1
	Kipf-none: acc = 0.459
	Kipf-both: acc = 0.649
	A-GCNN-False: acc = 0.865


/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


	A-GCNN-True: acc = 0.270
	H-GCNN-False: acc = 0.838
	H-GCNN-True: acc = 0.892
	W-GCN-A-False: acc = 0.865
	W-GCN-A-True: acc = 0.270
- RUN: 2
	Kipf-none: acc = 0.405


In [ ]:
table_comp2

,mean accs,med accs,std
Kipf-none,0.436757,0.432432,0.036405
Kipf-both,0.628108,0.621622,0.015815
A-GCNN-False,0.855135,0.864865,0.016887
A-GCNN-True,0.270270,0.270270,0.000000
H-GCNN-False,0.861622,0.864865,0.020682
H-GCNN-True,0.796757,0.810811,0.060250
W-GCN-A-False,0.836757,0.837838,0.029089
W-GCN-A-True,0.270270,0.270270,0.000000
